In [1]:
!pip install python-dotenv

In [2]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.6/602.6 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 78.6 MB/s eta 0:00:0

In [3]:
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [4]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.1 MB/s eta 0:00:00


In [5]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 10.0 MB/s eta 0:00:00


In [6]:
import os
import re   # 정규표현식 활용
from dotenv import load_dotenv
import pandas as pd   # 데이터 처리
import chromadb
from chromadb import Client
from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.schema import Document

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
os.chdir('/content/drive/MyDrive/NH_BigData')

In [9]:
os.getcwd()

'/content/drive/MyDrive/NH_BigData'

In [10]:
filepath = os.path.join(os.getcwd(), 'data', 'slopes_about_normalized.csv')
df = pd.read_csv(filepath)
print(df.head())

  tck_iem_cd    bse_dt  mkt_pr_tot_amt   trd_cst  slope_tco_avg_eal_pls_1d  \
0         AA  20240528        0.000371  0.004107                       NaN   
1         AA  20240529        0.000377  0.002793                  0.498236   
2         AA  20240530        0.000382  0.003127                  0.498397   
3         AA  20240531        0.000380  0.002416                  0.498423   
4         AA  20240603        0.000377  0.002062                  0.498849   

   slope_tco_avg_eal_pls_5d  slope_tco_avg_eal_pls_14d  \
0                       NaN                        NaN   
1                       NaN                        NaN   
2                       NaN                        NaN   
3                       NaN                        NaN   
4                       NaN                        NaN   

   future_slope_tco_avg_eal_pls_1d  future_slope_tco_avg_eal_pls_5d  \
0                         0.501764                         0.547192   
1                         0.501603      

In [11]:
# 임베딩 모델 설정
emb_model = HuggingFaceEmbeddings(model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS")

<ipython-input-11-0f627b8d811f>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb_model = HuggingFaceEmbeddings(model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or data

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77794 entries, 0 to 77793
Data columns (total 55 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   tck_iem_cd                        77794 non-null  object 
 1   bse_dt                            77794 non-null  int64  
 2   mkt_pr_tot_amt                    77794 non-null  float64
 3   trd_cst                           77794 non-null  float64
 4   slope_tco_avg_eal_pls_1d          76517 non-null  float64
 5   slope_tco_avg_eal_pls_5d          71409 non-null  float64
 6   slope_tco_avg_eal_pls_14d         59916 non-null  float64
 7   future_slope_tco_avg_eal_pls_1d   76517 non-null  float64
 8   future_slope_tco_avg_eal_pls_5d   71409 non-null  float64
 9   future_slope_tco_avg_eal_pls_14d  59916 non-null  float64
 10  slope_tco_avg_pft_rt_1d           76517 non-null  float64
 11  slope_tco_avg_pft_rt_5d           71409 non-null  float64
 12  slop

In [13]:
df.columns

Index(['tck_iem_cd', 'bse_dt', 'mkt_pr_tot_amt', 'trd_cst',
       'slope_tco_avg_eal_pls_1d', 'slope_tco_avg_eal_pls_5d',
       'slope_tco_avg_eal_pls_14d', 'future_slope_tco_avg_eal_pls_1d',
       'future_slope_tco_avg_eal_pls_5d', 'future_slope_tco_avg_eal_pls_14d',
       'slope_tco_avg_pft_rt_1d', 'slope_tco_avg_pft_rt_5d',
       'slope_tco_avg_pft_rt_14d', 'future_slope_tco_avg_pft_rt_1d',
       'future_slope_tco_avg_pft_rt_5d', 'future_slope_tco_avg_pft_rt_14d',
       'slope_lss_ivo_rt_1d', 'slope_lss_ivo_rt_5d', 'slope_lss_ivo_rt_14d',
       'future_slope_lss_ivo_rt_1d', 'future_slope_lss_ivo_rt_5d',
       'future_slope_lss_ivo_rt_14d', 'slope_ifw_act_cnt_1d',
       'slope_ifw_act_cnt_5d', 'slope_ifw_act_cnt_14d',
       'future_slope_ifw_act_cnt_1d', 'future_slope_ifw_act_cnt_5d',
       'future_slope_ifw_act_cnt_14d', 'slope_ofw_act_cnt_1d',
       'slope_ofw_act_cnt_5d', 'slope_ofw_act_cnt_14d',
       'future_slope_ofw_act_cnt_1d', 'future_slope_ofw_act_cnt_5d',
   

In [16]:
import tqdm
ids = []
metadatas = []
embeddings = []
documents = []

feature_columns = [col for col in df.columns if col not in ['tck_iem_cd', 'bse_dt']]

for row in tqdm.tqdm(df[:20000].iterrows()):
    index = row[0]
    bse_dt = row[1].bse_dt
    tck_iem_cd = row[1].tck_iem_cd

    metadata = {
        "date": bse_dt,
        "stock": tck_iem_cd
    }

    # feature 값들을 메타데이터에 추가
    for feature in feature_columns:
        metadata[feature] = str(row[1][feature])

    # 임베딩을 위한 쿼리 문자열 생성
    query = f"Stock: {row[1].tck_iem_cd} Date: {row[1].bse_dt} " + " ".join([f"{feature}: {row[1][feature]}" for feature in feature_columns])

    # Document 객체 생성
    doc = Document(page_content=query, metadata=metadata)

    # Document 객체 생성
    doc = Document(page_content=query, metadata=metadata)
    documents.append(doc)

20000it [00:09, 2049.88it/s]


In [20]:
# Chroma DB에 저장
DB_PATH = "/content/chroma_db"
docsearch = Chroma.from_documents(documents, emb_model,
                                  persist_directory=DB_PATH,
                                  collection_name="stocks")

In [21]:
# Llama 모델 및 토크나이저 로드
model_name = "allganize/Llama-3-Alpha-Ko-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

In [50]:
# 프롬프트 템플릿 정의
prompt_template = """
당신은 사용자의 투자 정보와 제공된 개별 주식 종목 정보(context)를 바탕으로 사용자의 투자 성향을 판별하는 전문가입니다.
투자 성향 판별 기준 가이드에 따라 사용자의 투자 성향을 분류하여 설명하세요.출력에서는 가이드의 내용을 포함시키지 마세요.

사용자 투자 정보:
{question}

context(주식 종목 정보):
{context}

결과는 다음 형식을 따르세요:

- Risk: 성향 (비율) | 성향 (비율)
  설명...

- Crowd Psychology: 성향 (비율) | 성향 (비율)
  설명...

- Interest: 성향 (비율) | 성향 (비율)
  설명...

- Volatility: 성향 (비율) | 성향 (비율)
  설명...

최종 NHTI 결과:
NHTI 유형: 성향 코드 (예: BGPS)
설명...


예시1:
입력:
- Stock : AAPL, Date : 20240716, 수량 : 100주
- Stock : AA, Date : 20240607, 수량 : 150주

출력:
Risk: B (Balanced) 85% | A (Aggressive) 15%
시가총액이 매우 높고 거래대금 또한 큰 APPL을 100주 가지고 있고, 시가총액이 낮고 거래대금 또한 낮은 AA를 150주 가지고 있으며 총 자산대비 AAPL의 비율이 90% 이상 차지하고 있습니다.
한 종목에 많은 투자가 이루어지고 있지만 우량주의 투자한 것으로 보아 낮은 리스크를 선호하며, 안정적인 자산에 투자하는 성향이 강한 것으로 보입니다.
Crowd Psychology: G (Guided) 70% | I (Independent) 30%
AAPL과 AA를 매수했을 당시 손실 투자자 비율, 당사평균평가손익, 당사평균수익율의 변화 수치를 분석한 결과 군중의 심리에 큰 영향을 받지만 약간의 독립적인 주관을 가지는 성향을 가지고 있습니다.
Interest: P (Popular) 75% | U (Unconventional) 25%
AAPL을 매수했을 당시의 경우 신규 매수 계좌 수, 종목 조회 건수, 관심 종목 등록 건수가 모두 높고, 거래대금 또한 활발함. AA의 같은 경우 매수시점 당시 신규 매수 계좌 수, 종목 조회 건수, 관심 종목 등록 건수가 모두 AAPL대비 낮긴 하지만 변화량에서 증가하는 추세를 보인다.
대중의 관심이 집중된 종목, 또는 관심이 늘어날 때에 따라가는 성향이 강하다고 평가할 수 있음.
Volatility: S (Secure) 87% | X (Exploratory) 13%
AA의 경우 NATR 지표가 높게 나타났다. AAPL의 경우 NATR 지표가 그렇게 높지 않은 것으로 나타났고 AAPL의 투자 자산 분포가 높은 것으로 보아 변동성이 크지 않는 투자를 선호하는 것으로 보입니다.

최종 NHTI 결과
NHTI 유형: BGPS (Balanced / Guided / Popular / Secure)
사용자는 BGPS 유형으로 수익이 적더라도 안전하게 분산투자를 선호하고, 상품에 대한 군중들의 시선을 어느 정도 의식하지만 자신만의 주관 또한 존재하며,
주식시장에서 인기있는 또는 뜨고있는 매물을 선호하지만 변동성이 작은 자산에서 안정을 얻는 경향이 강합니다.
"""
PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [54]:
# 파이프라인 생성

pipe = pipeline(
    "text-generation",
    do_sample=True,
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=3000,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

# LangChain용 LLM 객체 생성
llm = HuggingFacePipeline(pipeline=pipe)

# RetrievalQA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [55]:
# 챗봇 함수
def chatbot(query):
    result = qa_chain({"query": query})
    return result["result"]

In [56]:
# 대화 루프
print("주식 데이터 챗봇입니다. 질문해 주세요. (종료하려면 'quit' 입력)")
while True:
    user_input = input("사용자: ")
    if user_input.lower() == 'quit':
        break
    response = chatbot(user_input)
    print("챗봇:", response)

print("챗봇을 종료합니다.")

주식 데이터 챗봇입니다. 질문해 주세요. (종료하려면 'quit' 입력)
사용자: - Stock : AAPL, Date : 20240716, 수량 : 1주 - Stock : AA, Date : 20240607, 수량 : 200주
챗봇: 
당신은 사용자의 투자 정보와 제공된 개별 주식 종목 정보(context)를 바탕으로 사용자의 투자 성향을 판별하는 전문가입니다.
투자 성향 판별 기준 가이드에 따라 사용자의 투자 성향을 분류하여 설명하세요.출력에서는 가이드의 내용을 포함시키지 마세요.

사용자 투자 정보: 
- Stock : AAPL, Date : 20240716, 수량 : 1주 - Stock : AA, Date : 20240607, 수량 : 200주

context(주식 종목 정보):
Stock: AFCG Date: 20240528 mkt_pr_tot_amt: 8.391651581129499e-06 trd_cst: 1.711899472503166e-05 slope_tco_avg_eal_pls_1d: nan slope_tco_avg_eal_pls_5d: nan slope_tco_avg_eal_pls_14d: nan future_slope_tco_avg_eal_pls_1d: 0.5001825537970168 future_slope_tco_avg_eal_pls_5d: 0.5440280672469666 future_slope_tco_avg_eal_pls_14d: 0.3438257311443033 slope_tco_avg_pft_rt_1d: nan slope_tco_avg_pft_rt_5d: nan slope_tco_avg_pft_rt_14d: nan future_slope_tco_avg_pft_rt_1d: 0.550432601527492 future_slope_tco_avg_pft_rt_5d: 0.3551748624178945 future_slope_tco_avg_pft_rt_14d: 0.4661566725033729 slope_lss_ivo_rt_1d: nan s